In [9]:
import sys
import numpy as np

In [10]:
def read_off(filename):
    with open(filename) as fn:
        line = fn.readline()
        if (''.join(line.split()) != 'OFF'):
            print(filename + ' is not an OFF file')
            sys.exit(0)
        
        line = fn.readline()
        nvfe = np.asarray(line.split(),dtype=int)
        vertices = np.zeros((nvfe[0],3))
        triangles = np.zeros((nvfe[1],3),dtype=int)
        for i in range(nvfe[0]):
            line = fn.readline()
            vertices[i] = np.asarray(line.split(),dtype=float)
            
        for i in range(nvfe[1]):
            line = fn.readline()
            triangles[i] = np.asarray(line.split(),dtype=int)[1:]
            
        return(vertices,triangles)

In [11]:
lv,lt = read_off('socket.off')

In [12]:
print(lt[:10])

[[  6 398   7]
 [  6 400 398]
 [  6   5 400]
 [400   5 402]
 [403 402 405]
 [ 11 405  12]
 [ 11 403 405]
 [ 11  10 403]
 [403  10 401]
 [400 401 398]]


In [13]:
print(lv[:10])

[[-82.071068   7.071068  25.      ]
 [-75.        10.        25.      ]
 [-67.928932   7.071068  25.      ]
 [-65.         0.        25.      ]
 [-67.928932  -7.071068  25.      ]
 [-75.       -10.        25.      ]
 [-82.071068  -7.071068  25.      ]
 [-85.         0.        25.      ]
 [-82.071068  -7.071068 -25.      ]
 [-75.       -10.       -25.      ]]


In [20]:
def ccw(x1,y1,x2,y2,x3,y3):
    tmp=(x2-x1)*(y3-y1)-(y2-y1)*(x3-x1)
    if tmp>0:
        return True
    elif tmp<0:
        return False
    else:
        raise ValueError('The three points are on the same line')

corner_table = [[None]*7 for i in range(len(lt)*3)]
for idx,vertices in enumerate(lt[:10]):
    

6
6
6
400
403
11
11
11
403
400


5088